# Create or Update Seafloor Site Metadata File

This notebook goes through the steps necessary to update or create a new site metadata file. 

***
## Import packages

In [ ]:
from datetime import datetime
import json
import ipywidgets as widgets
from ipywidgets import HBox
from IPython.display import Image, display
import functools
import os
import sys

from es_sfgtools.utils.metadata_generator import Site, start_and_end_dates, buttons, import_site
from es_sfgtools.utils.gage_data import get_file_from_gage_data

***
## Choosing the appropriate site metadata file

Using the options below, you may either `create` or `update` a site metadata file. If the site already exists in the earthscope archive, the metadata file should be accessible on https://gage-data.earthscope.org/archive/seafloor. 

Only choose 1 of the 3 options.

### Option 1: Create new site metadata file

Use this cell below to create a new site metadata file. If you wish to use a previously created site file, skip this section and go to the next section.

In [ ]:
# ----------------------- Update these values ----------------------- 
# Add site names and networks
names = ['AAAA']                 # list of names used for this site, including the 4 char ID
networks = ['aleutian']          # list of networks this site is a part of              

# Add time of origin
time_of_origin = datetime(year=1900, month=1, day=1, 
                          hour=0, minute=0, second=0)

# height
local_geoid_height = 0

# ---------------------- Do not update code below ---------------------
# Create site object
site_class = Site(names=names, 
                  networks=networks,
                  time_of_origin=time_of_origin,
                  local_geoid_height=local_geoid_height)
print(json.dumps(site_class.site, indent=2))

### Option 2: Pull existing site metadata from local file path
Use the cell below to load an existing site metadata file to update

In [ ]:
# Pull in site metadata from json file

json_file_path = 'site.json'
site_class = import_site(json_file_path)
print(json.dumps(site_class.site, indent=2))

### Option 3: Pull existing site metadata directly from GAGE data
Use the cell below to load an existing site metadata file from the GAGE data archive

In [ ]:
# ----------------------- Update these values -----------------------
gage_http_file_link = ''
destination_directory = './gage_data'   # directory where the files will be downloaded to (default is the current directory/gage_data)
token_path = '.'                        # directory where the token will be stored (default is the current directory)


# ---------------------- Do not update code below ---------------------
get_file_from_gage_data(url=gage_http_file_link, 
                        dest_dir=destination_directory)

file_name = os.path.basename(gage_http_file_link)
site_class = import_site(os.path.join(destination_directory, file_name))
print(json.dumps(site_class.site, indent=2))

***
## Optional additions and updates
Below are optional additions and updates you can make to the site metadata file. You do not need to go in order to update the file. Options include..
* Reference Frames
* Campaigns 
* Survey Vessels (waveglider, gnss receiver/antennas, transducers, transcievers)
* Benchmarks 
* Transponders
***

## Instructions for adding each data type
1. Input values in "Update these Values" section
   * For a new section, enter all known values, including a start and end date. If a start and end date is not changed from "1900", then it will not be added into the json
   * For updating an existing section, update only the values you wish to change. Leave all other fields empty or as they are.
2. Run the cell and check the printed JSON input is correct.
3. Use the buttons provided to add or update the campaign.
***

## Reference frame

In [ ]:
reference_frame = {}
# ----------------------- Update these values ----------------------- 
reference_frame['name'] = ""         # e.g ITRF2008

# Enter the start/end date & time of the reference frame
ref_start = datetime(year=1900, month=1, day=1,
                          hour=0, minute=0, second=0)
ref_end = datetime(year=1900, month=1, day=1,
                        hour=0, minute=0, second=0)

# ---------------------- Do not update code below ---------------------
reference_frame = start_and_end_dates(start_date=ref_start, 
                                      end_date=ref_end, 
                                      dict_to_update=reference_frame)
# Button actions
output = widgets.Output()
display(widgets.HBox([buttons['new_ref'], buttons['existing_ref']]), output) 
buttons['new_ref'].on_click(functools.partial(site_class.new_top_level_group, "referenceFrames", reference_frame, output))
buttons['existing_ref'].on_click(functools.partial(site_class.existing_ref_frame, reference_frame, output))

## Campaign

In [ ]:
campaign = {}
# ----------------------- Update these values ----------------------- 

# Generate the campaign name
campaign_year = ""              # Year of campaign - eg. 2024
campaign_interval = ""          # alphabetical interval of campaign - 1st: a, 2nd: b, etc..
waveglider_code = ""            # 4 digit waveglider code

# Enter information known about the people and vessels involved in the campaign
campaign['type'] = ""                         # type of campaign: deploy | measure 
campaign['launchVesselName'] = ""             # launch vessel name used in campaign
campaign['recoveryVesselName'] = ""           # recovery vessel name used in campaign
campaign['principalInvestigator'] = ""        # PI name 
campaign['cruiseName'] = ""                   # Name of cruise
campaign['technicianName'] = ""               # technician name
campaign['technicianContact'] = ""            # technician contact information (email/phone)

# Enter the start/end date & time of the campaign # todo are these normally in UTC time??? 
campaign_start = datetime(year=1900, month=1, day=1, 
                          hour=0, minute=0, second=0)
campaign_end = datetime(year=1900, month=1, day=1, 
                        hour=0, minute=0, second=0)


# ----------------------- Do not update code below ----------------------- 
campaign_name = campaign_year + "_" + campaign_interval.upper() + "_" + waveglider_code.upper()
campaign['name'] = campaign_name
campaign['surveys'] = []

campaign = start_and_end_dates(start_date=campaign_start, 
                                      end_date=campaign_end, 
                                      dict_to_update=campaign)

# Button actions 
output = widgets.Output()
display(widgets.HBox([buttons['new_campaign'], buttons['existing_campaign']]), output)
buttons['new_campaign'].on_click(functools.partial(site_class.new_top_level_group, "campaigns", campaign, output))
buttons['existing_campaign'].on_click(functools.partial(site_class.existing_campaign, campaign, output))


## Benchmark

In [ ]:
benchmark = {'dropPointLocation': {}, 'aPrioriLocations': {}}
# ----------------------- Update these values ----------------------- 

# Benchmark information
benchmark['name'] = ""                 # "IVB1-1"
benchmark['benchmarkID'] = ""          # stamped on name

# Enter start and end of benchmark, if only a start date, do not update the end date
benchmark_start = datetime(year=1900, month=1, day=1,
                          hour=0, minute=0, second=0)
benchmark_end = datetime(year=1900, month=1, day=1,
                          hour=0, minute=0, second=0)

# Drop points
benchmark['dropPointLocation']['latitude'] = 0         # latitude of benchmark drop point
benchmark['dropPointLocation']['longitude'] = 0        # longitude of benchmark drop point
benchmark['dropPointLocation']['elevation'] = 0        # elevation of benchmark drop point

# Apriori locations  # todo this is a list.. should it be?
benchmark['aPrioriLocations']['latitude'] = 0          # latitude of benchmark drop point
benchmark['aPrioriLocations']['longitude']  = 0        # longitude of benchmark drop point
benchmark['aPrioriLocations']['elevation']  = 0        # elevation of benchmark drop point

# todo should there be a start and end apriori

# ----------------------- Do not update code below ----------------------- 
benchmark = start_and_end_dates(start_date=benchmark_start, 
                                end_date=benchmark_end, 
                                dict_to_update=benchmark)
benchmark['transponders'] = []

# Button actions
output = widgets.Output()
display(widgets.HBox([buttons['new_benchmark'], buttons['existing_benchmark']]), output)
buttons['new_benchmark'].on_click(functools.partial(site_class.new_top_level_group, "benchmarks", benchmark, output))
buttons['existing_benchmark'].on_click(functools.partial(site_class.existing_benchmark, benchmark, output))

## Benchmark transponder

In [ ]:
# ----------------------- Update these values ----------------------- 
transponder = {}
 
# Change this if you didn't previously add a benchmark or wish to add to a different benchmark
transponder_benchmark_name = benchmark['name']

transponder['uid'] = ""                       # UID?
transponder['model'] = ""                     # transponder model number
transponder['serialNumber'] = ""              # transponder serial number
transponder['batteryCapacity'] = ""           # e.g 4 Ah
transponder['address'] = ""                   # e.g 5209
transponder['tat'] = ""                       # TAT in ms

 # Change the campaign name if you didn't previously add a campaign or wish to add to a different campaign
transponder['deploymentCampaignName'] = "" #campaign_name      
transponder['recoveryCampaignName'] = "" #campaign_name

# Start and end date
transponder_start = datetime(year=1900, month=1, day=1,
                          hour=0, minute=0, second=0)
transponder_end = datetime(year=1900, month=1, day=1,
                          hour=0, minute=0, second=0)

# Add any notable events if applicable
notes = ""

# Optional:  Add battery voltage
add_battery_voltage = False          # set to True if adding a voltage 
voltage_date = datetime(year=1900, month=1, day=1,
                          hour=0, minute=0, second=0)
voltage_value = 0                    # e.g 12.0

# Optional: Add extra sensor
add_sensor = False                   # set to True if adding a sensor
sensor_type = ""                     # e.g "pressure"
sensor_model_number = ""             # pressure sensor model number
sensor_serial_number = ""            # pressure sensor serial number


# ----------------------- Do not update code below ----------------------- 
if add_battery_voltage:
    print("adding battery voltage")
    transponder['batteryVoltage'] = [{}]
    transponder['batteryVoltage'][0]['date'] = voltage_date.strftime('%Y-%m-%dT%H:%M:%S')
    transponder['batteryVoltage'][0]['voltage'] = voltage_value
    
if add_sensor:
    print("adding sensor")
    transponder['extraSensors'] = [{}]
    transponder['extraSensors'][0]['type'] = sensor_type
    transponder['extraSensors'][0]['model'] = sensor_model_number
    transponder['extraSensors'][0]['serialNumber'] = sensor_serial_number

transponder = start_and_end_dates(start_date=transponder_start, 
                                end_date=transponder_end, 
                                dict_to_update=transponder)

# Button actions
output = widgets.Output()
display(widgets.HBox([buttons['new_transponder'], buttons['existing_transponder']]), output)
buttons['new_transponder'].on_click(functools.partial(site_class.add_transponder_to_benchmark, transponder_benchmark_name, transponder, output))
buttons['existing_transponder'].on_click(functools.partial(site_class.existing_transponder, transponder_benchmark_name, transponder, output))
    

## Campaign Survey

In [ ]:
survey = {}
notes = {}
# ----------------------- Update these values ----------------------- 
 # Change this if you didn't previously add a campaign or wish to add to a different campaign
survey_campaign_name = campaign_name       

# Update this value to the survey ID if you wish to update an existing survey
survey['id'] = ""                 # campaign_name + interval

survey['type'] = ""               # e.g cirle drive | fixed point
survey['vesselName'] = ""         # Vessel name 
survey['benchmarkIDs'] = []       # List of benchmark IDs in survey['IVB1-1', 'IVB1-2', 'IVB1-3']

# Enter the start and end date/time of the survey
survey_start = datetime(year=1900, month=1, day=1, 
                        hour=0, minute=0, second=0)
survey_end = datetime(year=1900, month=1, day=1, 
                        hour=0, minute=0, second=0)

# Add any notes or commands
add_notes = False                       # Change to True if you wish to add notes for the survey
notes['commands'] = ''                  # log of commands for a given survey
notes['notes'] = ''                     # i.e any strange data flags

    
# ----------------------- Do not update code below -----------------------    
survey = start_and_end_dates(start_date=survey_start, 
                                end_date=survey_end, 
                                dict_to_update=survey)
if survey['id']:
    print("Survey will be updated to: " + survey['id'])

if add_notes:
    survey['notes'] = notes

# Button actions
output = widgets.Output()
display(widgets.HBox([buttons['new_survey'], buttons['existing_survey']]), output)
buttons['new_survey'].on_click(functools.partial(site_class.new_survey, survey, survey_campaign_name, output))
buttons['existing_survey'].on_click(functools.partial(site_class.existing_survey, survey, survey_campaign_name, survey['id'], output))

## Survey Vessel
Add the primary suvey vessel (e.g waveglider, boat, etc..). If the survey vessel already exists but is being used in a new survey, the ID will be updated.


In [ ]:
survey_vessel = {}
# ----------------------- Update these values ----------------------- 

survey_id = ''                             # Survey ID the vessel was used in
survey_vessel["type"] = ''                 # waveglider
survey_vessel["name"] = ''                 # e.g GLDR
survey_vessel['model'] = ''                # waveglider model number
survey_vessel['serialNumber'] = ''         # waveglider serial number
    
# ----------------------- Do not update code below ----------------------- 
print("Adding or updating: \n" + json.dumps(survey_vessel, indent=2))
output = widgets.Output()
display(widgets.HBox([buttons['new_survey_vessel'], buttons['existing_survey_vessel']]), output)
buttons['new_survey_vessel'].on_click(functools.partial(site_class.add_primary_survey_vessel, survey_vessel, survey_id, output))

## Survey Vessel Equipment

### INS Payload

In [ ]:
ins_payload = {}
# ----------------------- Update these values ----------------------- 
 # Change this if you didn't previously add a survey vessel or wish to add to a different vessel
ins_payload_survey_vessel = survey_vessel['name']
survey_id = ''                            # Survey ID the INS was used in

ins_payload['type'] = ''                  # type of INS 
ins_payload['model'] = ''                 # INS model number
ins_payload['serialNumber' ] = ''         # INS serial number


# ----------------------- Do not update code below -----------------------     
# Button actions
print("Adding or updating: \n" + json.dumps(ins_payload, indent=2))
output = widgets.Output()
display(widgets.HBox([buttons['new_payload']]),  output)
buttons['new_payload'].on_click(functools.partial(site_class.add_survey_vessel_equipment, ins_payload_survey_vessel, "insPayloads", ins_payload, 
                                                  survey_id, output))


### GNSS receiver

In [ ]:
gnss_receiver = {}
# ----------------------- Update these values -----------------------  
# Change this if you didn't previously add a survey vessel OR wish to add to a different vessel
gnss_survey_vessel = survey_vessel['name']
survey_id = ''


gnss_receiver['type'] = ''                  # type of gnss receiver e.g "trimble netrs"
gnss_receiver['model'] = ''                 # receiver model number
gnss_receiver['serialNumber' ] = ''         # receiver serial number
gnss_receiver['firmwareVersion'] = ''       # receiver firmware version

# Enter the start and end date/time of the gnss receiver
receiver_start = datetime(year=1900, month=1, day=1, 
                        hour=0, minute=0, second=0)
receiver_end = datetime(year=1900, month=1, day=1, 
                        hour=0, minute=0, second=0)


# ----------------------- Do not update code below -----------------------  
print("Adding or updating: \n" + json.dumps(gnss_receiver, indent=2))
# Button actions
output = widgets.Output()
display(widgets.HBox([buttons['new_receiver']]), output)
buttons['new_receiver'].on_click(functools.partial(site_class.add_survey_vessel_equipment, gnss_survey_vessel, "gnssReceivers", gnss_receiver, 
                                                   survey_id, output))


### GNSS antenna

In [ ]:
gnss_antenna = {}
# ----------------------- Update these values -----------------------  
# Change this if you didn't previously add a survey vessel or wish to add to a different vessel
gnss_survey_vessel = ''
survey_id = ''

gnss_antenna['type'] = ''                  # type of gnss antenna e.g "trimble"
gnss_antenna['model'] = ''                 # antenna model number
gnss_antenna['serialNumber' ] = ''         # antenna serial number
gnss_antenna['radomeSerialNumber'] = ''    # antenna radome serial number


# ----------------------- Do not update code below -----------------------     
print("Adding or updating: \n" + json.dumps(gnss_antenna, indent=2))
    
# Button actions
output = widgets.Output()
display(widgets.HBox([buttons['new_antenna']]), output)
buttons['new_antenna'].on_click(functools.partial(site_class.add_survey_vessel_equipment, gnss_survey_vessel, "gnssAntennas", gnss_antenna, 
                                                  survey_id, output))


### Acoustic Transducer

In [ ]:
transducer = {}
# ----------------------- Update these values -----------------------  
# Change this if you didn't previously add a survey vessel or wish to add to a different vessel
transducer_survey_vessel = survey_vessel['name']
survey_id = ''


transducer['type'] = ''                 # transducer model number
transducer['serialNumber'] = ''         # transducer serial number
transducer['frequency'] = ''            # frequency e.g MF/LMF


# ----------------------- Do not update code below ----------------------- 
print("Adding or updating: \n" + json.dumps(transducer, indent=2))
    
# Button actions
output = widgets.Output()
display(widgets.HBox([buttons['new_transducer']]), output)
buttons['new_transducer'].on_click(functools.partial(site_class.add_survey_vessel_equipment, transducer_survey_vessel, "acousticTransducer", transducer, 
                                                     survey_id, output))

### Acoustic Transceiver

In [ ]:
transceiver = {}
# ----------------------- Update these values -----------------------  
# Change this if you didn't previously add a survey vessel or wish to add to a different vessel
transceiver_survey_vessel = survey_vessel['name']
survey_id = ''


transceiver['type'] = ''                      # transceiver model number
transceiver['serialNumber'] = ''              # transceiver serial number
transceiver['frequency'] = ''                 # frequency e.g MF/LMF
transceiver['triggerDelay'] = ''              # delay seconds
transceiver['delayIncludedInTWTT'] = 'false'  # transceiver frequency e.g MF/LMF


# ----------------------- Do not update code below -----------------------  
print("Adding or updating: \n" + json.dumps(transceiver, indent=2))
    
# Button actions
output = widgets.Output()
display(widgets.HBox([buttons['new_transceiver']]), output)
buttons['new_transceiver'].on_click(functools.partial(site_class.add_survey_vessel_equipment, transceiver_survey_vessel, "acousticTransceiver", transceiver, 
                                                      survey_id, output))

### Acoustic to Transducer Offsets

In [ ]:
atd = {}
# ----------------------- Update these values -----------------------  
# Change this if you didn't previously add a survey vessel or wish to add to a different vessel
atd_survey_vessel = survey_vessel['name']        # 
survey_id = ''

atd['x'] = ''
atd['y'] = ''
atd['z'] = ''

# ----------------------- Do not update code below -----------------------  
print("Adding or updating: \n" + json.dumps(atd, indent=2))
    
# Button actions
output = widgets.Output()
display(widgets.HBox([buttons['new_atd']]), output)
buttons['new_atd'].on_click(functools.partial(site_class.add_survey_vessel_equipment, atd_survey_vessel, "atdOffsets", atd, 
                                                survey_id, output))


# Write out JSON file
This step can be completed at any time.

* First check the contents...

In [ ]:
print(json.dumps(site_class.site, indent=2))

## Export Site to file

In [ ]:
file_path = "./site.json"                            # Export file path you wish to store
site_class.export_site(file_path)